In [14]:
import pickle
from collections import OrderedDict

import numpy as np
from periodictable import elements

In [37]:
from exfor_tools import curate, quantities, reaction
from jitr import rmatrix
from jitr.utils import kinematics
from jitr.xs.elastic import DifferentialWorkspace
import elm

In [16]:
with open("nn_elastic_data.pkl", "rb") as f:
    nn_data = pickle.load(f)

In [17]:
with open("pp_elastic_data.pkl", "rb") as f:
    pp_data = pickle.load(f)

In [18]:
all_data = list(pp_data.values()) + list(nn_data.values())
all_data_by_entry, sys_uncertainties_by_entry = (
    curate.cross_reference_entry_systematic_err(all_data)
)

In [19]:
for entry_id, entries in all_data_by_entry.items():
    for entry in entries:
        for measurement in entry.measurements:
            if measurement.systematic_offset_err > 0:
                # TODO
                measurement.systematic_offset_err = 0

In [20]:
from IPython.display import Latex, Math, display  # just for fun

targets = sorted(
    list(set(list(nn_data.keys()) + list(pp_data.keys()))), key=lambda x: x[1] + x[0]
)
display(
    Math(
        ", ".join(
            [f"\,^{{{target[0]}}} \\rm{{{elements[target[1]]}}}" for target in targets]
        )
    )
)

<IPython.core.display.Math object>

In [21]:
entry = pp_data[(40, 20)].data["dXS/dA"].entries

In [22]:
measurement_groups = curate.categorize_measurements_by_energy(
    pp_data[(40, 20)].data["dXS/dA"].entries
)

In [23]:
constraints = []
models = []
solver = rmatrix.Solver(40)
for measurement_group in measurement_groups:
    for m in measurement_group:
        constraints.append(
            elm.calibration.ReactionDistribution(
                quantity="dXS/dA",
                measurement=m,
            )
        )
        models.append(
            elm.calibration.ElasticModel(
                quantity="dXS/dA",
                reaction=pp_data[(40, 20)].reaction,
                Elab=m.Einc,
                angles_rad_vis=np.linspace(0.01, np.pi, 100),
                angles_rad_constraint=m.x * np.pi / 180,
                core_solver=solver,
                lmax=30,
            )
        )

In [39]:
def el_model(
    workspace: DifferentialWorkspace,
    params: OrderedDict,
):
    rxn = workspace.reaction
    kinematics=workspace.kinematics
    assert rxn.projectile.A == 1
    (
        isoscalar_params,
        isovector_params,
        spin_orbit_params,
        coul_params,
        asym_factor,
    ) = elm.calculate_parameters(
        projectile=tuple(rxn.projectile),
        target=tuple(rxn.target),
        E=kinematics.Ecm,
        Ef=rxn.target.Efp if rxn.projectile.Z > 0 else rxn.target.Efn,
        params=params,
    )
    return workspace.xs(
        interaction_central=elm.central_plus_coulomb,
        interaction_spin_orbit=elm.spin_orbit,
        args_central=(
            workspace.projectile,
            asym_factor,
            isoscalar_params,
            isovector_params,
            coul_params,
        ),
        args_spin_orbit=spin_orbit_params,
    )